In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow import keras
import seaborn as sns
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

In [2]:
emotion_df=pd.read_csv('Emotions.csv')
violence_df=pd.read_csv('Violence.csv')
hate_df=pd.read_csv('Hatespeech.csv')

In [3]:
emotion_df.head()

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4


In [4]:
violence_df.head()

,Tweet_ID,tweet,type
0,ID_0022DWKP,Had a dream i got raped last night. By a guy i...,sexual_violence
1,ID_00395QYM,he thought the word raped means sex and told m...,sexual_violence
2,ID_003EOSSF,She NOT TALKING TO ME I WAS RAPED BY 2 MEN 1 M...,sexual_violence
3,ID_004BBHOD,I was sexually abused for 3 years at age 4 to ...,sexual_violence
4,ID_004F7516,Chessy Prout can do better by telling the trut...,sexual_violence


In [5]:
hate_df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [6]:
emotion_df.drop(columns=['Unnamed: 0'], inplace=True)
violence_df.drop(columns=['Tweet_ID'], inplace=True)
hate_df=hate_df[['tweet','class']]

In [7]:
emotion_df.columns,violence_df.columns,hate_df.columns

(Index(['text', 'label'], dtype='object'),
 Index(['tweet', 'type'], dtype='object'),
 Index(['tweet', 'class'], dtype='object'))

In [8]:
violence_df.rename(columns={'tweet':'text','type':'label'},inplace=True)
hate_df.rename(columns={'tweet':'text','class':'label'},inplace=True)

In [9]:
emotion_df.columns,violence_df.columns,hate_df.columns

(Index(['text', 'label'], dtype='object'),
 Index(['text', 'label'], dtype='object'),
 Index(['text', 'label'], dtype='object'))

In [10]:
emotion_df.isna().sum(), violence_df.isna().sum(), hate_df.isna().sum()

(text     0
 label    0
 dtype: int64,
 text     0
 label    0
 dtype: int64,
 text     0
 label    0
 dtype: int64)

In [11]:
emotion_df.shape, violence_df.shape, hate_df.shape

((416809, 2), (39650, 2), (24783, 2))

In [12]:
emotion_df['label'].value_counts()

label
1    141067
0    121187
3     57317
4     47712
2     34554
5     14972
Name: count, dtype: int64

In [13]:
violence_df['label'].value_counts()

label
sexual_violence                 32648
Physical_violence                5946
emotional_violence                651
economic_violence                 217
Harmful_Traditional_practice      188
Name: count, dtype: int64

In [14]:
hate_df['label'].value_counts()

label
1    19190
2     4163
0     1430
Name: count, dtype: int64

In [15]:
e_df=pd.DataFrame()
for i in  range(len(emotion_df['label'].unique())):
    subset=emotion_df[emotion_df['label']==i].sample(n=2000,random_state=42)
    e_df=pd.concat([e_df,subset])

In [16]:
e_df.shape

(12000, 2)

In [17]:
emotion_df=e_df.copy()

In [18]:
emotion_df['label'].value_counts()

label
0    2000
1    2000
2    2000
3    2000
4    2000
5    2000
Name: count, dtype: int64

In [19]:
sexual_violence=violence_df[violence_df['label']=='sexual_violence'].sample(n=4998, random_state=42)
violence_df=violence_df[violence_df['label']!='sexual_violence']

In [20]:
violence_df=pd.concat([violence_df,sexual_violence], axis=0)

In [21]:
violence_df['label'].value_counts()

label
Physical_violence               5946
sexual_violence                 4998
emotional_violence               651
economic_violence                217
Harmful_Traditional_practice     188
Name: count, dtype: int64

In [22]:
offensive_speech=hate_df[hate_df['label']==1].sample(n=6407, random_state=42)
hate_df=hate_df[hate_df['label']!=1]

In [23]:
hate_df=pd.concat([offensive_speech,hate_df], axis=0)

In [24]:
hate_df['label'].value_counts()

label
1    6407
2    4163
0    1430
Name: count, dtype: int64

In [25]:
hate_df.shape,emotion_df.shape, violence_df.shape

((12000, 2), (12000, 2), (12000, 2))

In [26]:
label_encoder=LabelEncoder()
violence_df['label']=label_encoder.fit_transform(violence_df['label'])

In [27]:
violence_df

,text,label
6,"My Husband Beats Me Frequently, Wife Tells Cou...",1
29,"Best thing for me to do, is remain silent when...",1
30,"My husband will never beat me, Bambam denies r...",1
33,"theyre like, i just wanna be a baby maker with...",1
35,"I was in England for a week, the longest I’ve ...",1
...,...,...
25368,i was 13 and had just been raped. the police t...,4
13942,So bad.She/he don't know how blessed they are ...,4
34704,I was Hide and he was Yoshi. We were Hideyoshi...,4
18626,It actually sickens me that he's so smug abt '...,4


In [28]:
violence_df['label'].unique()

array([1, 3, 0, 2, 4])

In [29]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ambik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [30]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ambik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ambik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [31]:
stop_words=set(stopwords.words('english'))

In [32]:
len(stop_words)

179

In [33]:
def remove_stopwords(text):
    all_words=nltk.word_tokenize(text)
    filtered_words=[word for word in all_words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [34]:
emotion_df['text']=emotion_df['text'].apply(remove_stopwords)
violence_df['text']=violence_df['text'].apply(remove_stopwords)
hate_df['text']=hate_df['text'].apply(remove_stopwords)

In [35]:
emotion_df.head()

,text,label
133243,ive learned surround women lift leave feeling ...,0
88501,already feel crappy upset situation doesnt help,0
131379,feel like lost mourned moved past tears relati...,0
148369,could write whole lot im feeling crappy dont t...,0
134438,always seem feel inadequate,0


In [36]:
tokenizer=Tokenizer()

In [37]:
tokenizer.fit_on_texts(pd.concat([emotion_df['text'],violence_df['text'],hate_df['text']]))

In [38]:
emotion_sequence=tokenizer.texts_to_sequences(emotion_df['text'])
violence_sequence=tokenizer.texts_to_sequences(violence_df['text'])
hate_sequence=tokenizer.texts_to_sequences(hate_df['text'])

In [39]:
emotion_df['text'].iloc[2]

'feel like lost mourned moved past tears relationship'

In [40]:
emotion_sequence[2:3]

[[1, 5, 321, 11854, 1207, 422, 1093, 385]]

In [41]:
max_length=50
emotion_padded=pad_sequences(emotion_sequence,maxlen=max_length,padding='post')
violence_padded=pad_sequences(violence_sequence,maxlen=max_length,padding='post')
hate_padded=pad_sequences(hate_sequence,maxlen=max_length,padding='post')

In [42]:
emotion_padded[2:3]

array([[    1,     5,   321, 11854,  1207,   422,  1093,   385,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0]])

In [43]:
emotion_labels=np.array(emotion_df['label'])
violence_labels=np.array(violence_df['label'])
hate_labels=np.array(hate_df['label'])

In [44]:
hate_input=hate_padded
violence_input=violence_padded
emotion_input=emotion_padded

In [45]:
emotion_input_layer=keras.layers.Input(shape=(max_length,),name='emotion_input')
violence_input_layer=keras.layers.Input(shape=(max_length,),name='violence_input')
hate_input_layer=keras.layers.Input(shape=(max_length,),name='hate_input')

In [46]:
embedding_layer=keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1,output_dim=128)

In [47]:
emotion_embedding=embedding_layer(emotion_input_layer)
violence_embedding=embedding_layer(violence_input_layer)
hate_embedding=embedding_layer(hate_input_layer)

In [48]:
shared_lstm=keras.layers.LSTM(64,return_sequences=True)

In [49]:
emotion_lstm=shared_lstm(emotion_embedding)
violence_lstm=shared_lstm(violence_embedding)
hate_lstm=shared_lstm(hate_embedding)

In [50]:
shared_pooling=keras.layers.GlobalAveragePooling1D()
shared_dropout=keras.layers.Dropout(0.5)

In [51]:
emotion_features=shared_dropout(shared_pooling(emotion_lstm))
violence_features=shared_dropout(shared_pooling(violence_lstm))
hate_features=shared_dropout(shared_pooling(hate_lstm))

In [52]:
len(emotion_df['label'].unique()), len(violence_df['label'].unique()), len(hate_df['label'].unique())

(6, 5, 3)

In [53]:
emotion_output=keras.layers.Dense(6,activation='softmax', name='emotion_output')(emotion_features)
violence_output=keras.layers.Dense(5,activation='softmax', name='violence_output')(violence_features)
hate_output=keras.layers.Dense(3,activation='softmax',name='hate_output')(hate_features)

In [54]:
model=keras.models.Model(inputs=[emotion_input_layer,violence_input_layer,hate_input_layer],
                       outputs=[emotion_output,violence_output,hate_output])

model.compile(optimizer='adam',
             loss={'emotion_output':'sparse_categorical_crossentropy',
                  'violence_output':'sparse_categorical_crossentropy',
                  'hate_output':'sparse_categorical_crossentropy'
                  },
             metrics={'emotion_output':'accuracy',
                     'violence_output':'accuracy',
                     'hate_output':'accuracy'
                     })

In [55]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ emotion_input       │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ violence_input      │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hate_input          │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 50, 128)   │  5,350,656 │ emotion_input[0]… │
│ (Embedding)         │                   │            │ violence_input[0… │
│                     │                   │            │ hate_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 50, 64)    │     49,408 │ embedding[0][0],  │
│                     │                   │            │ embedding[1][0],  │
│                     │                   │            │ embedding[2][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ lstm[0][0],       │
│ (GlobalAveragePool… │                   │            │ lstm[1][0],       │
│                     │                   │            │ lstm[2][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ global_average_p… │
│                     │                   │            │ global_average_p… │
│                     │                   │            │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emotion_output      │ (None, 6)         │        390 │ dropout[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ violence_output     │ (None, 5)         │        325 │ dropout[1][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hate_output (Dense) │ (None, 3)         │        195 │ dropout[2][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,400,974 (20.60 MB)

 Trainable params: 5,400,974 (20.60 MB)

 Non-trainable params: 0 (0.00 B)

In [56]:
model.fit(x={'emotion_input':emotion_input,
            'violence_input':violence_input,
            'hate_input':hate_input
            },
         y={'emotion_output':emotion_labels,
           'violence_output':violence_labels,
           'hate_output':hate_labels
           },
         epochs=10,
         batch_size=4
         )

Epoch 1/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 196s 64ms/step - emotion_output_accuracy: 0.2501 - emotion_output_loss: 1.6776 - hate_output_accuracy: 0.6653 - hate_output_loss: 0.7621 - loss: 2.8770 - violence_output_accuracy: 0.8316 - violence_output_loss: 0.4374
Epoch 2/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 193s 64ms/step - emotion_output_accuracy: 0.8583 - emotion_output_loss: 0.4699 - hate_output_accuracy: 0.8773 - hate_output_loss: 0.3741 - loss: 0.8847 - violence_output_accuracy: 0.9863 - violence_output_loss: 0.0407
Epoch 3/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 199s 66ms/step - emotion_output_accuracy: 0.9464 - emotion_output_loss: 0.1769 - hate_output_accuracy: 0.9304 - hate_output_loss: 0.1942 - loss: 0.3803 - violence_output_accuracy: 0.9982 - violence_output_loss: 0.0093
Epoch 4/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 201s 67ms/step - emotion_output_accuracy: 0.9665 - emotion_output_loss: 0.1014 - hate_output_accuracy: 0.9694 - hate_output_loss: 0.0931 - loss: 0.2016 - violence_output_accuracy

In [57]:
prediction=model.predict({'emotion_input':emotion_input,
                         'violence_input':violence_input,
                         'hate_input':hate_input})

375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step


In [58]:
prediction

[array([[9.9999082e-01, 3.0554295e-07, 2.2053836e-08, 6.5727781e-06,
         2.2629335e-06, 1.7340255e-08],
        [9.9999332e-01, 9.9137161e-08, 1.3767607e-08, 6.2677445e-06,
         3.4016662e-07, 9.3483798e-09],
        [9.9997425e-01, 1.2596621e-06, 2.5053529e-07, 2.3268674e-05,
         7.4771208e-07, 2.3066730e-07],
        ...,
        [2.2234965e-06, 6.7730798e-05, 1.8152022e-07, 1.4185264e-05,
         7.6083589e-04, 9.9915481e-01],
        [2.7024765e-08, 3.0500414e-07, 3.0333500e-09, 2.1402310e-07,
         7.3834167e-06, 9.9999201e-01],
        [1.5249270e-06, 7.5421904e-06, 1.2599931e-07, 4.1478029e-06,
         1.5019233e-05, 9.9997163e-01]], dtype=float32),
 array([[1.49864821e-12, 1.00000000e+00, 1.03048736e-10, 9.28713961e-14,
         3.60873305e-13],
        [1.14756793e-09, 9.99999881e-01, 7.67319293e-08, 6.64897803e-10,
         1.28626731e-09],
        [4.93646408e-11, 1.00000000e+00, 4.97080466e-09, 4.02376431e-11,
         9.66512703e-11],
        ...,
      

In [59]:
emotion_pred=np.argmax(prediction[0],axis=1)
violence_pred=np.argmax(prediction[1],axis=1)
hate_pred=np.argmax(prediction[2],axis=1)